In [8]:
from config import DB_USERNAME, DB_PASSWORD 
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [10]:
username = DB_USERNAME # use username in your database connection 
password = DB_PASSWORD # use password in your database connection 
hostname = "localhost"
port = "5432"
database_name = "m1_db"

In [11]:
# Create the engine
db_string = f"postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{database_name}"
engine = create_engine(db_string)

In [12]:
try:
    with engine.begin() as connection:
        connection.execute(text('DROP VIEW IF EXISTS JPD;'))
        connection.execute(text('DROP TABLE IF EXISTS a;'))
        connection.execute(text('DROP TABLE IF EXISTS b;'))
        connection.execute(text('DROP TABLE IF EXISTS c;'))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [13]:
try:
    with engine.begin() as connection:
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.a
            (
                id INTEGER NOT NULL PRIMARY KEY,
                a boolean NOT NULL,
                p_a REAL NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO a(id, a, p_a)
                    VALUES (1, false, 0.75), (2, true, 0.25)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [14]:
try:
    with engine.begin() as connection:
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.b
            (
                id INTEGER NOT NULL PRIMARY KEY,
                a_id INTEGER NOT NULL,
                FOREIGN KEY (a_id) REFERENCES a(id),
                b boolean NOT NULL,
                p_b_a REAL NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO b(id, a_id, b, p_b_a)
                    VALUES (1, 1, false, 0.2), (2, 1, true, 0.8),
                    (3, 2, false, 0.9), (4, 2, true, 0.1)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

In [16]:
try:
    with engine.begin() as connection:
        connection.execute(text('''
            CREATE TABLE IF NOT EXISTS public.c
            (
                id INTEGER NOT NULL PRIMARY KEY,
                b_id INTEGER NOT NULL,
                FOREIGN KEY (b_id) REFERENCES b(id),
                c boolean NOT NULL,
                p_c_b REAL NOT NULL
            )
        '''))
        connection.execute(text('''
                    INSERT INTO c(id, b_id, c, p_c_b)
                    VALUES (1, 1, false, 1), 
                    (2, 2, false, 0.25),(3, 2, true, 0.75), 
                    (4, 3, false, 1),
                    (5, 4, false, 0.25), (6, 4, true, 0.75)
                '''))
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")
            

In [17]:
try:
    with engine.begin() as connection:
        result = connection.execute(text("SELECT * FROM a")).fetchall()
        for row in result:
            print(row)
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

(1, False, 0.75)
(2, True, 0.25)


In [18]:
try:
    with engine.begin() as connection:
        result = connection.execute(text("SELECT * FROM b")).fetchall()
        for row in result:
            print(row)
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

(1, 1, False, 0.2)
(2, 1, True, 0.8)
(3, 2, False, 0.9)
(4, 2, True, 0.1)


In [19]:
try:
    with engine.begin() as connection:
        result = connection.execute(text("SELECT * FROM c")).fetchall()
        for row in result:
            print(row)
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")


(1, 1, False, 1.0)
(2, 2, False, 0.25)
(3, 2, True, 0.75)
(4, 3, False, 1.0)
(5, 4, False, 0.25)
(6, 4, True, 0.75)


In [22]:
try:
    with engine.begin() as connection:
        connection.execute(text('''
            CREATE OR REPLACE VIEW JPD
            AS
            SELECT a.a, b.b, c.c, a.p_a * b.p_b_a * c.p_c_b AS prob
            FROM a
            JOIN b ON a.id = b.a_id
            JOIN c ON c.b_id = b.id  
             ''')) 
        print('Created view JPD.')
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

Created view JPD.


In [21]:
try:
    with engine.begin() as connection:
        result = connection.execute(text("SELECT * FROM JPD")).fetchall()
        for row in result:
            print(row)
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

(False, False, False, 0.15)
(False, True, False, 0.15)
(False, True, True, 0.45000002)
(True, False, False, 0.225)
(True, True, False, 0.00625)
(True, True, True, 0.01875)


In [24]:
try:
    with engine.begin() as connection:
        print('getting posterior probability of a, b given c is true')

        result = connection.execute(text('''
            SELECT a, b, Prob / (SELECT SUM(Prob) FROM JPD WHERE c = true) as prob1
            FROM JPD 
            WHERE c = true 
        '''))

        for row in result:
            print(row)
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

getting posterior probability of a, b given c is true
(False, True, 0.96)
(True, True, 0.04)


In [26]:
try:
    with engine.begin() as connection:
        print('getting most probable explanation of a, b given c is true')

        result = connection.execute(text('''
            SELECT a, b 
            FROM JPD 
            WHERE c = true
            AND prob = (SELECT Max(prob) from JPD)
        '''))

        for row in result:
            print(row)
except SQLAlchemyError as e:
    print(f"An error occurred: {str(e)}")

getting most probable explanation of a, b given c is true
(False, True)
